### The following code uses variational form U3 (three parameters) for single qubit state, and COBYLA optimizer to minimize the cost<br>
$$ C(p_0, p_1) = (p_0 \cdot S_1 - p_1 \cdot S_2)^2 $$
#### where<br>
$S_1$ - area formed by smallest and second largest sides<br>
$S_2$ - area formed by second smallest and largest sides<br>
#### If given sides may form a rectangle, areas become equal and state with even $p_0, p_1$ probabilities will minimize the cost

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
import numpy as np
import scipy as sp
from qiskit.algorithms.optimizers import COBYLA
from scipy.optimize import minimize


backend = Aer.get_backend("aer_simulator")

In [3]:
# Prepare single qubit state with given three parameters
def prepare_variational(params) -> QuantumCircuit: 
    """Prepare single qubit state using U3 variational form. Params: theta,phi,lambda"""
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [4]:
# Convert measurement counts to probability distribution
def counts_to_distr(counts) -> dict:
    n_shots = sum(counts.values())
    return {int(k, 2): v/n_shots for k, v in counts.items()}

In [5]:
# Create cost function(params), given sides. This is curried
def make_objective(sides: list):
    def cost(params):
        circuit = prepare_variational(params)
        result = backend.run(circuit).result()
        distr = counts_to_distr(result.get_counts())
        s_sorted = sorted(sides)
        
        #Cost is difference of areas of 2 rectangles:
        #Rectangle 1 sides: (smallest,2nd largest)
        #Rectangle 2 sides: (2nd smallest, largest)
        #If given sides are repeated, even distribution will correspond to minimal cost
        return (distr.get(0,0)*(s_sorted[0]*s_sorted[2]) - distr.get(1,0)*(s_sorted[1]*s_sorted[3]))**2
    return cost

In [18]:
"""Use COBYLA optimizer to find params, that minimize the cost. 
   Then prepare the state using found params and get the distribution.
   If the distribution is close to even, one may form a rectangle
   """
def is_rectangle(a: int, b: int, c: int, d: int) -> bool:
    
    params = np.random.rand(3)
    #Minimize the cost using COBYLA
    result = minimize(fun=make_objective([a,b,c,d]), x0=params, method='COBYLA', 
                  options={'maxiter': 5000, 'disp': False}, tol=0.0001)
    
    # Obtain the output distribution using the final parameters
    qc = prepare_variational(result.x)
    counts = backend.run(qc, shots=10000).result().get_counts()
    output_distr = counts_to_distr(counts)
    
    #Optimal params should correspond to even distribution (like |x> or |-x> state)
    return np.isclose(output_distr.get(0,0), output_distr.get(1,0), atol=0.1)

In [19]:
is_rectangle(6,9,9,6)

True

In [20]:
is_rectangle(5,7,4,7)

False

In [21]:
def is_rectangle_classic(a: int, b: int, c: int, d: int) -> bool:
    return len(set([a,b,c,d])) == 2

In [25]:
test_rectangles = np.random.randint(20, size=(10, 4))
test = [is_rectangle_classic(a,b,c,d) == is_rectangle(a,b,c,d) for [a,b,c,d] in test_rectangles]
assert all(test)